<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Подготовим-признаки-и-матрицы" data-toc-modified-id="Подготовим-признаки-и-матрицы-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Подготовим признаки и матрицы</a></span></li><li><span><a href="#Создадим-модель-LinearRegression" data-toc-modified-id="Создадим-модель-LinearRegression-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Создадим модель LinearRegression</a></span><ul class="toc-item"><li><span><a href="#Обучим-на-изначальных-признаках" data-toc-modified-id="Обучим-на-изначальных-признаках-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Обучим на изначальных признаках</a></span></li><li><span><a href="#Обучим-на-новых-признаках" data-toc-modified-id="Обучим-на-новых-признаках-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Обучим на новых признаках</a></span></li><li><span><a href="#Сравним-качество" data-toc-modified-id="Сравним-качество-4.2.3"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Сравним качество</a></span></li></ul></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании. Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

# План работы

1) Загрузим и изучим данные.

2) Ответим на вопрос:
 * Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
   * a. Изменится. Приведите примеры матриц.
   * b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
   
3) Предложим алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

4) Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится

**Обоснование:** Представим новую матрицу как результат умножения на обратимую матрицу $X_1 = XP$

$w_1 = ((XP)^T XP)^{-1} (XP)^T y$

Используем свойства матриц:

$A(BC) = (AB)C$

$(AB)^{-1} = A^{-1} B^{-1}$

$(AB)^T = A^T B^T$

$(A^T)^{-1} = (A^{-1})^T$

$(ABC)^{-1} = C^{-1}B^{-1}A^{-1}$

$AA^{-1} = E$

$AE = EA = A$


Выведем по ним нашу формулу:

$w_1 = ((XP)^T XP)^{-1} (XP)^T y$ =>

$w_1 = (P^T(X^TX)P)^{-1})(XP)^T y$ =>

$w_1 = (P^T(X^TX)P)^{-1})P^TX^T y$ =>

$ w_1 = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^T y $ =>

$ w_1 = P^{-1}(X^TX)^{-1}EX^T y $ =>

$ w_1 = P^{-1}w$

Подставим получившиеся значения для формулы предсказаний

$a_1 = X_1w_1$   =>   $a_1 = XPP^{-1}w$   =>   $a_1 = Xw$

**Вывод:** Наши предсказания не изменятся

## Алгоритм преобразования

**Алгоритм**

* Генерируем обратную квадратную матрицу
* Умножаем матрицу на сгенерированную
* Вычислить качество модели 
* Сравнить качество модели полученной матрицы с качеством исходной

**Обоснование**

Исходя из полученных ранее формул качество не должно отличаться

## Проверка алгоритма

### Подготовим признаки и матрицы

In [6]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']
# создадим признаки

In [7]:
matrix = features.values
matrix
#сделаем матрицу из наших признаков

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

In [8]:
matrix_random = np.random.normal(size=(4, 4))
matrix_random
#сгенерируем матрицу 4x4

array([[-1.32373162,  0.53385348, -0.83133966, -1.08747399],
       [ 0.85142882, -0.43311157, -0.52033417,  0.63511069],
       [ 1.90530893,  0.17322142, -0.24015406, -0.69599732],
       [ 1.73027373,  0.78281145, -1.17864802,  0.56230694]])

In [9]:
inmatrix_random = np.linalg.inv(matrix_random)
inmatrix_random
#сделаем обратимую матрицу

array([[-0.18403608,  0.01034665,  0.34189357,  0.05557694],
       [-0.1975088 , -1.28398379, -0.25120037,  0.757329  ],
       [-0.57113266, -0.83849522,  0.05692764, -0.08702213],
       [-0.35589003, -0.00191366, -0.58300772,  0.37065596]])

In [10]:
new_matrix = matrix@inmatrix_random
new_matrix
#умножим основную матрицу на сгенерированную обратную

array([[-28336.81757971, -41641.99803595,   2813.07079257,
         -4284.82085087],
       [-21712.48226518, -31921.88369772,   2151.11223151,
         -3271.63309243],
       [-11999.51355462, -17645.6352437 ,   1188.19570456,
         -1805.50215717],
       ...,
       [-19366.05903216, -28450.67161255,   1923.65709475,
         -2934.16226364],
       [-18681.63478735, -27447.03687778,   1854.60040756,
         -2827.79481857],
       [-23194.05605127, -34078.84922648,   2303.98760511,
         -3511.46697151]])

In [11]:
new_features = pd.DataFrame(new_matrix, columns = features.columns)
new_features

,Пол,Возраст,Зарплата,Члены семьи
0,-28336.817580,-41641.998036,2813.070793,-4284.820851
1,-21712.482265,-31921.883698,2151.112232,-3271.633092
2,-11999.513555,-17645.635244,1188.195705,-1805.502157
3,-23821.091266,-34992.218347,2367.441514,-3612.177537
4,-14912.276633,-21920.666558,1479.119781,-2250.016764
...,...,...,...,...
4995,-20395.677885,-29970.234887,2024.117250,-3084.743463
4996,-29934.422422,-43980.807125,2973.884704,-4533.839691
4997,-19366.059032,-28450.671613,1923.657095,-2934.162264
4998,-18681.634787,-27447.036878,1854.600408,-2827.794819


### Создадим модель LinearRegression

In [12]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T@X)@X.T@y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

#### Обучим на изначальных признаках

In [13]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)

#### Обучим на новых признаках

In [14]:
model_new = LinearRegression()
model_new.fit(new_features, target)
predictions_new = model_new.predict(new_features)

#### Сравним качество

In [15]:
print('Качество R2 на изначальных признаках:', r2_score(target, predictions))
print('Качество R2 на новых признаках:', r2_score(target, predictions_new))

Качество R2 на изначальных признаках: 0.42494550286668
Качество R2 на новых признаках: 0.4249455028666642


# Вывод:

Метрики качества отличаются незначительно, значит вычисления верны